# Śledzenie obiektów

## Wstęp
W erze cyfrowej, w obliczu rosnącej lawinowo ilości danych wideo, zdolność do ich automatycznego rozpoznawania i interpretowania staje się kluczowa w wielu dziedzinach – od bezpieczeństwa publicznego po autonomiczne pojazdy. Technologie oparte na głębokim uczeniu rewolucjonizują sposób, w jaki przetwarzamy informacje wizualne. Kluczowym wyzwaniem jest tu detekcja i śledzenie obiektów na filmach wideo.

Celem tego zadania jest opracowanie algorytmu, który będzie w stanie analizować sekwencje ruchów w grze "trzy kubki". Uczestnicy mają za zadanie określić końcową pozycję kubków po serii ruchów, korzystając z analizy statycznych obrazów z każdej klatki nagrania.

In [32]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# Poniższe funkcje ułatwiają pracę z dostarczonymi danymi
# W kolejnych komórkach zobaczysz przykłady ich użycia
from utils.utils import get_level_info, get_video_data, display_video, download_and_replace_data

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

images, coordinates, _, path_to_images = get_video_data(level=3,video_id=0,dataset="example")
display_video(images,rescale=0.7,FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

## Zadanie 3: Zbuduj rozwiązanie od zera

W tym zadaniu podejdziesz do problemu identyfikacji obiektów na filmach wideo od podstaw. Poprzednie zadania wymagały użycia gotowych informacji o lokalizacji obiektów w klatkach. Tym razem będziesz musiał stworzyć algorytm, który będzie operować bezpośrednio na nieoznaczonych obrazach, co pozwoli na pełniejsze zrozumienie i opracowanie własnego systemu detekcji obiektów.

Napisz algorytm, który poradzi sobie ze zbiorem danych `level_3` bez podanych prostokątów ograniczających.

## Pliki zgłoszeniowe
Tylko ten notebook zawierający **kod** oraz **krótki raport** opisujący Twoje rozwiązanie (do 300 słów). Miejsce na raport znajdziesz na końcu tego notebooka.

## Ograniczenia
- Twoja funkcja powinna zwracać predykcje w maksymalnie 5 minut używając Google Colab bez GPU.

## Uwagi i wskazówki
- Testuj swoje rozwiązanie na zbiorze plików wideo `level_3`.
- **Skuteczność modelu**: przetestuj skuteczność modelu na zbiorze walidacyjnym używając dostarczonej przez nas funkcji **submission_script**, umieść ten wynik w raporcie.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to podzadanie możesz zdobyć pomiędzy 0 i 0.5 punktów. Zdobędziesz 0 punktów jeśli Twoje accuracy na zbiorze testowym będzie poniżej 30%. Jeśli będzie równe 100%, otrzymasz 0.5 punktu. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością metryki.

# Kod startowy

In [33]:
# Poniższe biblioteki są wystarczające do wykonania wszystkich zadań
# Jeśli jednak chcesz użyć innych, sprawdź czy są dostępne na serwerze (requirements.txt)
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import IPython.display
import json
import PIL
import sklearn as sk
import gdown
import os

In [34]:
# funkcja pomocnicza do ładowania danych
images, _, _, _ = get_video_data(level=3,video_id=0,dataset="example")

with open(os.path.join(os.getcwd(),'example_tracks','tracks_3_0.json'), 'r') as f:
    tracks = json.load(f)

for key in tracks.keys():
    tracks[key] = [tuple(el) for el in tracks[key]]

# funkcja pomocnicza do wyświetlania danych
display_video(images,
                tracks=tracks,
                rescale=0.7,
                FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

In [35]:
# Pobieranie danych do podzadań 1, 2 i 3 (około ~646Mb), skrypt będzie wykonywał się parę minut
# Wystarczy, że pobierzesz dane tylko raz. Na serwerze sprawdzającym dane będą już pobrane,
# struktura plików będzie identyczna jak tutaj
if not FINAL_EVALUATION_MODE:
    download_and_replace_data()


In [36]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# funkcja pomocnicza do testowania algorytmu
# Zwróć uwagę na to że funkcja ta działa inaczej niż w poprzednich podzadaniach
# algorytm przyjmuje jako argument listę obrazków, a nie koordynaty
def submission_script(algorithm,level,verbose=False,dataset="valid"):
    num_videos, _ = get_level_info(level=level,dataset=dataset)
    correct = []
    exception_messages = set()
    for video_number in range(num_videos):
        images, _, target, _ = get_video_data(level=level,video_id=video_number,dataset=dataset)
        try:
            prediction = algorithm(images)
            if tuple(target) == tuple(prediction):
                correct.append(1)
            else:
                correct.append(0)
            if verbose:
                print(f"Video: animation_{str(video_number).zfill(4)}")
                print(f"Prediction: {prediction}")
                print(f"Target:     {target}")
                print(f"Score: {tuple(target) == tuple(prediction)}", end='\n\n')
        except Exception as e:
            correct.append(0)
            exception_messages.add(str(e))
    if verbose:
        print(f"Accuracy: {np.mean(correct)}")
        print(f"Correctness: {correct}")
    return np.sum(correct) / num_videos, correct, exception_messages

# Twoje rozwiązanie

In [ ]:
# Hubert Jastrzębski - V LO Kraków

from sklearn.cluster import KMeans, MiniBatchKMeans
from itertools import permutations 
from statistics import median
from copy import deepcopy, copy
import math

np.random.seed(42)
torch.manual_seed(42)

In [38]:
def get_points(images): # nie zmieniaj nazwy funkcji
    points = np.array([])

    prev_clusters = [[155, 255], [210, 320], [290, 465]]
    for i in range(len(images)):
        coors = np.argwhere(np.array(images[i])[:,:,1] == 0)

        m_iter = (10 if i == 0 else 1)
        kmeans = KMeans(n_clusters=3, init=prev_clusters, max_iter=m_iter, n_init=1).fit(coors)
        points = np.append(points, kmeans.cluster_centers_)

        prev_clusters = kmeans.cluster_centers_

    points = points.reshape((-1, 3, 2))
    return points

In [39]:
def distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def your_algorithm_task_3(images): # nie zmieniaj nazwy funkcji
    points = get_points(images)
    
    indices_p = np.argsort(points[0][:, 0])
    points[0] = points[0][indices_p]

    for i in range(1, len(points)):
        p = copy(points[i])
        best_perm = [0, 1, 2]
        best_dist = math.inf
        
        for perm in list(permutations([0, 1, 2])):
            p_temp = [p[j] for j in perm]
            dist = sum(distance(points[i - 1][j], p_temp[j]) for j in [0, 1, 2])

            if dist < best_dist:
                best_dist = dist
                best_perm = perm

        points[i] = [points[i][j] for j in best_perm]

    rp = [points[-1][i][0] for i in [0, 1, 2]]
    res = [rp.index(i) for i in [min(rp), median(rp), max(rp)]]
    return res

In [ ]:
# Sprawdź jak działa Twój algorytm
if not FINAL_EVALUATION_MODE:
    accuracy, correctness, _ = submission_script(
        algorithm=your_algorithm_task_3,
        level=3,
        verbose=True,
        dataset="train")

In [41]:
# zapisz swój raport do zmiennej poniżej, abyśmy mogli go później automatycznie odczytać sprawdzaczką
raport_3 = \
"""
Raport z zadania:
znajduję czerwone piksele
używam algorytmu klasteryzacji do znalezienie trzech środków tych obiektów w każdej klatce
rozpatruję po kolei klatki
dla każdej klatki patrzę która permutacja środków daje najmniejszą sumę odległości 
"""